In [1]:
# Importing necessary libraries: Earth Engine (ee) and geemap for geospatial analysis
import ee
import geemap as geemap

# Initializing the Earth Engine API
geemap.ee_initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=0OjmQ-xD9kUb0LNWXos7rjPg0PqjPl55usAmy96ZIzU&tc=ksSH7getzazJWa61ilA9F5zhxwz3pDzboA41lT7moSs&cc=3WS6DuQqUZUr_iG94Z_XuuV8ktYF8CNYHSmr-RQZzXY

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXl25W4LDEBdmQgcA5_7rN8d4KDXxh0xzZXdq62WHRBVsRLu3DvhHWQ

Successfully saved authorization token.


In [12]:
# Defining various datasets from Earth Engine, including regions of interest, land use, and environmental features
roi = ee.FeatureCollection("users/ipam_flp/Analyses/Restoration_Carbon_Market/Vetor/assentamentos_pogo")
frequency = ee.Image("users/ipam_flp/Analyses/Restoration_Carbon_Market/Image/frequency_pogo_2008_2022")
land_use = ee.Image("projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_integration_v1").select('classification_2022')
fire_col2 = ee.Image("projects/mapbiomas-workspace/public/collection7_1/mapbiomas-fire-collection2-monthly-burned-coverage-1")
desforestation = ee.FeatureCollection("users/ipam_flp/Analyses/Restoration_Carbon_Market/prodes_2008_2022")
hidro_10 = ee.FeatureCollection("users/ipam_flp/Analyses/Restoration_Carbon_Market/hidro_10m").merge("users/ipam_flp/Analyses/Restoration_Carbon_Market/hidro_10m_lages")
sec_veg = ee.Image("projects/mapbiomas-workspace/public/collection8/mapbiomas_collection80_deforestation_secondary_vegetation_v1").select('classification_2022').eq(303).selfMask()
srtm = ee.Image("USGS/SRTMGL1_003")
region = roi.geometry().buffer(5000)

In [13]:
# Processing the hydrographic data to create a mask for water bodies
hidro_10 = ee.Image().paint(hidro_10, 0, 0.1).remap([0],[1]).clip(region)

# Creating a binary mask for forest land use from the land cover image
forest = ee.Image(1).mask(land_use.eq(3)).clip(roi)
sec_veg = sec_veg.updateMask(forest).clip(roi)

# Setting parameters for identifying significant forest areas based on connected pixel count
minArea = 100000;
maxSize = 500;

pixelCount = forest.connectedPixelCount(maxSize);

minPixelCount = ee.Image(minArea).divide(ee.Image.pixelArea());

forest_gt10ha = forest.updateMask(pixelCount.gte(minPixelCount));

# Filtering the deforestation data to focus on recent years
desforestation = desforestation \
  .filter(ee.Filter.gt('year', (2022 - 5)))

desforestation = ee.Image().paint(desforestation).remap([0],[1]).rename('constant')

# Identifying areas with high fire frequency
fire_more2 = ee.Image(1).mask(frequency.gt(2));

# Creating a composite image to remove areas affected by deforestation and frequent fires
removeArea = ee.ImageCollection([desforestation, fire_more2]).mosaic();

# Calculating the distance from significant forest areas
distanceForest = (
  forest_gt10ha
  .distance(kernel= ee.Kernel.euclidean(200), skipMasked= False)
  .rename('distance')
  .clip(roi)
  .selfMask()
)

# Processing elevation data to obtain slope and hillshade information
elevation = srtm.select('elevation').clip(region);
slope = ee.Terrain.slope(elevation).clip(region);
hillshade = ee.Terrain.hillshade(srtm).clip(region);

In [14]:
# Parameters for visualizing distance layer
imageVisParamDistance = {
  "opacity":1,
  "bands":["distance"],
  "min":2,
  "max":8.261297173761164,
  "palette":["bd0026","f03b20","fd8d3c","fecc5c","ffffb2"]
}

imageVisParamDistForest = {
  "opacity": 1,
  "bands": ["distance"],
  "max": 100,
  "palette": ["edf8e9","c7e9c0","a1d99b","74c476","31a354","006d2c"]
}

# Calculating the distance from hydrographic data
distanceHidro = hidro_10 \
  .distance(kernel= ee.Kernel.euclidean(200), skipMasked= False) \
  .rename('distance') \
  .clip(roi) \
  .selfMask()

# Visualization parameters for the hydrographic distance layer
imageVisParamDistHidro = {
  "opacity": 1,
  "bands": ["distance"],
  "max": 100,
  "palette": ["eff3ff","bdd7e7","6baed6", "3182bd","08519c"]
}

# Preparing the region of interest (ROI) for visualization
roi_img = ee.Image().paint(roi)

roi_img = roi_img \
  .remap([0], [1]) \
  .rename('constant')

potencialArea  = roi_img.updateMask(forest_gt10ha.unmask().Not())

# Loading a base map layer for visualization
basemap = ee.Image('users/ipam_flp/Analyses/Restoration_Carbon_Market/planet_roi_2023_ago').clip(region)

# Setting visualization parameters for the base map
visBasemap = {'bands':['R','G','B'],'min':150,'max':1000,'gamma':1.3}

In [15]:
# Function to get minimum and maximum values of an image
def get_min_max(image):
  min_max = image.reduceRegion(
      reducer = ee.Reducer.minMax(),
      geometry = region,
      scale = 30,
      maxPixels = 1e9
  )
  return min_max.getInfo()

In [16]:
# Retrieving min and max values
min_max_distanceForest = get_min_max(distanceForest)
min_max_distanceHidro = get_min_max(distanceHidro)
min_max_slope = get_min_max(slope)
min_max_fire_frequency = get_min_max(frequency)

print(min_max_distanceForest)
print(min_max_distanceHidro)
print(min_max_slope)
print(min_max_fire_frequency)

{'distance_max': 71.06335201775947, 'distance_min': 1}
{'distance_max': 59.64059020499378, 'distance_min': 1}
{'slope_max': 46.450927734375, 'slope_min': 0}
{'fire_frequency_max': 7, 'fire_frequency_min': 0}


In [17]:
# Normalizing images based on their min and max values
distance_forest_norm = distanceForest.unitScale(min_max_distanceForest['distance_min'], min_max_distanceForest['distance_max'])
distance_hidro_norm = distanceHidro.unitScale(min_max_distanceHidro['distance_min'], min_max_distanceHidro['distance_max'])
slope_norm = slope.unitScale(min_max_slope['slope_min'], min_max_slope['slope_max'])
fire_frequency_norm = frequency.unitScale(min_max_fire_frequency['fire_frequency_min'], min_max_fire_frequency['fire_frequency_max'])

# Defining weights for each variable in the restoration index
weight = {
    'slope': 0.20,
    'fire_frequency': 0.30,
    'distance_hidro': 0.25,
    'distance_forest': 0.25
    }

# Calculating the restoration index
index_restoration = distance_forest_norm.multiply(weight['distance_forest']) \
  .add(distance_hidro_norm.multiply(weight['distance_hidro'])) \
  .add(slope_norm.multiply(weight['slope'])) \
  .add(fire_frequency_norm.multiply(weight['fire_frequency'])) \
  .rename('restoration')

min_max_restoration = get_min_max(index_restoration )


In [18]:
# Setting up the map visualization
map = geemap.Map(height = 800)
map.centerObject(region)

# Adding various layers to the map for visualization
map.addLayer(basemap, visBasemap, '2023-08 planet')
map.addLayer(potencialArea , {'palete': '#0000ff'}, 'potencialArea')
map.addLayer(forest_gt10ha, {'palette': 'green'}, 'Forest')
map.addLayer(sec_veg, {'palette': 'yellow'}, 'sec_veg', False)
map.addLayer(distanceForest, imageVisParamDistForest, 'Distance - Forest')
map.addLayer(distance_forest_norm, imageVisParamDistForest, 'distance_forest_norm')
map.addLayer(distanceHidro, imageVisParamDistHidro, 'Distance - Hidro')
map.addLayer(hidro_10, {'palette': 'blue'}, 'Hidro_10', False)
map.addLayer(elevation, {'min':0, 'max': 300, 'palette': ['90EE90','FFFF00','FF0000']}, 'Raw SRTM', False)
map.addLayer(hillshade, {'min':150, 'max':255,}, 'Hillshade', False)
map.addLayer(slope, {'min':0, 'max':20, 'pallete': ['FFFFFF']},'Slope', False)
map.addLayer(index_restoration, {'min':0, 'max':100, 'pallete': ['FFFFFF']},'index_restoration', False)
